In [1]:
import os
import csv
import numpy as np
import scipy.io as sio
import pandas as pd
from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import RFE
from nilearn import connectome
from nilearn import datasets
import shutil

D:\Anaconda\envs\pytorch\lib\site-packages\nilearn\__init__.py:69: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


In [19]:
def feature_selection(matrix, labels, train_ind, fnum):
    """
        matrix       : feature matrix (num_subjects x num_features)
        labels       : ground truth labels (num_subjects x 1)
        train_ind    : indices of the training samples
        fnum         : size of the feature vector after feature selection
    return:
        x_data      : feature matrix of lower dimension (num_subjects x fnum)
    """

    estimator = RidgeClassifier()
    selector = RFE(estimator, n_features_to_select = fnum, step=100, verbose=1)
    print(np.shape(train_ind))
    print(train_ind)
    print("at ferature selection")

    featureX = matrix[train_ind, :]
    featureY = labels[train_ind]
    selector = selector.fit(featureX, featureY.ravel())
    x_data = selector.transform(matrix)

    print("Number of labeled samples %d" % len(train_ind))
    print("Number of features selected %d" % x_data.shape[1])

    return x_data

In [9]:
phenotype = r"Phenotypic_V1_0b_preprocessed1.csv"
def get_subject_score(subject_list, score):
    scores_dict = {}
    with open(phenotype) as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            if row['SUB_ID'] in subject_list:
                scores_dict[row['SUB_ID']] = row[score]
    return scores_dict

def Load_Raw_Data(atlas="aal"):

    subject_IDs = np.genfromtxt(r"subject_IDs.txt", dtype=str)
    labels = get_subject_score(subject_IDs, score='DX_GROUP')
    # adj = Reader.create_affinity_graph_from_scores(['SEX', 'SITE_ID'], subject_IDs)
    num_nodes = len(subject_IDs)
    y = np.zeros([num_nodes])
    for i in range(num_nodes):
        y[i] = int(labels[subject_IDs[i]])
    Label = y - 1
    Raw_Features = np.load(rf"{atlas}_871.npy")
    return Raw_Features, Label

In [31]:
features, label = Load_Raw_Data('dosenbach160')

In [32]:
features.shape, label.shape

((871, 12880), (871,))

In [33]:
from sklearn.model_selection import KFold,StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=1234, shuffle=True)

In [34]:
for train_index, test_index in skf.split(features, label):
    print(train_index)
    print(test_index)

[  0   1   2   3   5   6   7   8  11  14  16  19  20  21  23  25  27  28
  29  30  31  32  33  34  35  36  39  40  41  42  43  44  45  46  47  48
  49  51  52  53  54  55  57  58  59  60  61  62  63  64  66  67  68  69
  71  72  73  74  75  76  78  79  81  82  83  84  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 111 112
 113 114 117 118 119 121 122 123 124 126 127 129 130 131 132 134 135 136
 137 138 139 140 142 143 144 145 146 147 148 149 150 151 152 153 154 156
 157 158 159 160 162 163 164 165 166 168 169 170 172 173 174 175 177 178
 180 181 182 183 184 185 186 187 188 189 193 194 195 196 197 198 199 200
 201 203 207 208 209 210 211 212 213 216 217 218 219 220 221 223 224 225
 226 227 229 230 231 232 233 234 236 237 239 240 242 244 245 246 247 248
 249 250 251 252 254 255 256 257 258 259 260 261 262 263 264 265 266 267
 268 269 270 271 272 275 276 277 278 279 280 282 283 284 286 287 288 289
 290 291 292 293 294 295 296 297 298 300 302 303 30

In [35]:
feat_256_1 = feature_selection(features, label, train_index, 512)

(697,)
[  0   2   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  31  32  33  34  36  37  38  40  41
  42  43  44  46  48  49  50  52  53  54  55  56  58  60  61  62  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  83  85
  86  87  89  90  93  95  97  98 100 101 102 103 108 109 110 111 113 114
 115 116 117 118 119 120 121 122 123 124 125 126 128 129 130 132 133 134
 136 137 139 140 141 143 145 146 148 149 150 151 152 154 155 157 159 160
 161 162 164 165 166 167 168 171 172 173 174 175 176 177 178 179 182 183
 184 185 186 187 189 190 191 192 194 197 198 199 202 203 204 205 206 207
 209 210 211 213 214 215 216 217 218 219 220 222 223 227 228 229 230 231
 232 233 234 235 236 237 238 239 241 242 243 244 246 247 250 251 252 253
 256 257 258 260 261 262 263 264 265 267 268 269 270 271 272 273 274 275
 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 292 293 294
 295 296 297 298 299 301 302 304 306 307 308

In [36]:
feat_256_1

array([[ 7.27165432e-02,  5.95556390e-02, -1.55175387e-02, ...,
         6.05057334e-01,  8.46519650e-01,  9.88892879e-01],
       [ 1.16748904e-01,  3.61608457e-02,  4.06068101e-02, ...,
        -3.14109671e-02,  2.04885165e-01,  2.70966454e-01],
       [-3.84622303e-04,  3.12219709e-01,  2.48377851e-01, ...,
         5.94383427e-01,  7.15755909e-01,  6.50389289e-01],
       ...,
       [ 1.37901633e-01, -1.09017253e-01,  3.75124406e-02, ...,
         1.93237021e-01,  4.98885513e-01,  8.40615229e-01],
       [ 2.82753942e-01, -9.05262416e-02,  1.60940929e-01, ...,
         1.97756290e-01,  2.12653876e-01,  1.80203943e-01],
       [ 7.45733467e-02,  1.10805645e-01,  9.58529341e-02, ...,
         1.70503156e-01,  4.84675802e-01,  1.14018941e-01]])

In [37]:
np.save('dosenbach160_871_REF.npy', feat_256_1)